# **Project 2: LangChain RAG with Google Gemini Flash and Pinecone**

In [ ]:
!pip install -qU -U langchain-pinecone langchain-google-genai pinecone-client openai tqdm --quiet --upgrade langchain-text-splitters

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install pinecone-client

In [ ]:
!pip install pypdf

In [ ]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from tqdm import tqdm
from uuid import uuid4
import os
import time


# Initialize API keys and environment variables
pinecone_api_key = userdata.get('PINECONE_API_KEY')
google_api_key = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = google_api_key

# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)

index_name = "online-rag"
if index_name in pc.list_indexes().indexes:
    # Delete the existing index
    pc.delete_index(index_name)
    # Wait for the index to be deleted
    while index_name in pc.list_indexes().indexes:
        time.sleep(1)  # Wait for 1 second and check again
else:
    # List existing indexes
    existing_indexes = pc.list_indexes().indexes
    # If there are 5 or more serverless indexes, delete one
    if len(existing_indexes) >= 5:
        index_to_delete = existing_indexes[0].name
        print(f"Deleting index: {index_to_delete} to make space for new index.")
        pc.delete_index(index_to_delete)

pc.create_index(
    name=index_name,
    dimension=768,  # GOOGLE model dimension is 768
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)
index = pc.Index(index_name)

# Initialize embedding model
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Load and split documents
loader =  PyPDFLoader("toyota_corolla.pdf")
documents = loader.load()

# Split documents into chunks (300 characters per chunk with 50 characters overlap)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Create a vector store and add documents to Pinecone
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# Upsert vectors into Pinecone
vectors=[]
for i, doc in enumerate(tqdm(docs)): # Use enumerate to get the index 'i'
    vector = embeddings.embed_query(doc.page_content)
    # Generate a unique ID for each document
    doc_id = str(uuid4())
    # upsert each vector with a unique ID
    index.upsert(vectors=[(doc_id, vector, {"text": doc.page_content})])
    # Introduce a dynamic delay to stay within rate limits
    if (i + 1) % 150 == 0:
        time.sleep(90)


# Initialize LLM with Google Generative AI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    api_key=google_api_key,
    temperature=0.3,
    max_tokens=300,
)

# Set up the retriever for the QA chain
retriever = vector_store.as_retriever()

# Create a RetrievalQA chain using LangChain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "map_reduce" or "refine" can be used here
    retriever=retriever
)

# Define the query and get the response
query = "What are the interior features of Hybrid LE?"
response = qa_chain.run(query)
print("Question: ", query)
print("Response: ", response)

100%|██████████| 280/280 [02:28<00:00,  1.88it/s]


Question:  What are the interior features of Hybrid LE?
Response:  Based on the provided information, the Hybrid XLE has the following interior features:
*   Automatic climate control
*   4.2-in. Multi-Information Display (MID)
*   Fabric-trimmed seats include:
    *   60/40 split fold-down rear seat with center armrest
*   Multi-function in-key remote keyless entry system



# **Query the RAG System**

In [ ]:
# Define the query and get the response
query = "Which corolla model do you suggest in terms of exterior features?"
response = qa_chain.run(query)
print("Question: ", query)
print("Response: ", response)

Question:  Which corolla model do you suggest in terms of exterior features?
Response:  Based on the provided information, the Corolla's sporty design is a key exterior feature. The sport mesh grille and available 18-in. graphite-colored alloy wheels are specifically mentioned as features that enhance the car's style.

